In [71]:
#导入必要的包
import torch
import torch.nn as nn                           #模型工具包
import torch.optim as optim                     #优化器
from torchvision.transforms.v2 import ToTensor  #转换图像为张量
from sklearn.datasets import fetch_olivetti_faces     #数据集
from sklearn.model_selection import train_test_split  #数据拆分
from torch.utils.data import DataLoader, TensorDataset         #数据加载器 ，优化大量数据
from torch.utils.tensorboard import SummaryWriter   #可视化工具包
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau  #学习率调度器 ，动态调整学习率

# 1. 数据预处理模块
def prepare_data(test_size=0.2):
    # 加载原始数据
    faces = fetch_olivetti_faces(data_home='./data', shuffle=True, random_state=42)
    
    # 数据标准化和维度调整
    images = (faces.images - 0.5) / 0.5  # 标准化到[-1, 1]
    images = images.reshape(-1, 1, 4096)  # 调整为RNN输入形状(400, 1, 4096)
    labels = faces.target

    # 转换为PyTorch张量
    X = torch.tensor(images, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.long)

    # 数据集划分
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size,
        stratify=y,
        random_state=42
    )
    
    return (X_train, y_train), (X_test, y_test)

# 2. RNN模型定义
class FaceRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=4096,
            hidden_size=512,
            num_layers=2,
            batch_first=True,
            dropout=0.3
        )
        self.bn = nn.BatchNorm1d(512)
        self.fc = nn.Linear(512, 40)

    def forward(self, x):
        # x形状: [batch, 1, 4096]
        outputs, _ = self.rnn(x)
        last_out = outputs[:, -1, :]
        normalized = self.bn(last_out)
        return self.fc(normalized)

# 3. 训练流程
def train_model():
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    writer = SummaryWriter()

    # 准备数据
    (X_train, y_train), (X_test, y_test) = prepare_data()
    train_loader = DataLoader(TensorDataset(X_train, y_train), 
                             batch_size=32, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test),
                            batch_size=32)

    # 初始化模型
    model = FaceRNN().to(device)
    
    # 训练配置
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)

    best_acc = 0.0
    for epoch in range(100):
        # 训练阶段
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()

        # 验证阶段
        model.eval()
        correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                outputs = model(inputs).cpu()
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
        
        # 计算指标
        avg_loss = train_loss / len(train_loader)
        acc = 100 * correct / len(y_test)
        scheduler.step(acc)  # 动态调整学习率
        
        # 保存最佳模型
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_rnn.pth")

        # 记录日志
        writer.add_scalar('Loss/train', avg_loss, epoch)
        writer.add_scalar('Accuracy/test', acc, epoch)
        print(f"Epoch {epoch+1:03d} | Loss: {avg_loss:.4f} | Acc: {acc:.2f}%")

    writer.close()
    print(f"Best Accuracy: {best_acc:.2f}%")

if __name__ == "__main__":
    train_model()

Epoch 001 | Loss: 2.5655 | Acc: 71.25%
Epoch 002 | Loss: 0.8244 | Acc: 93.75%
Epoch 003 | Loss: 0.2537 | Acc: 93.75%
Epoch 004 | Loss: 0.1108 | Acc: 96.25%
Epoch 005 | Loss: 0.0527 | Acc: 96.25%
Epoch 006 | Loss: 0.0321 | Acc: 96.25%
Epoch 007 | Loss: 0.0144 | Acc: 96.25%
Epoch 008 | Loss: 0.0095 | Acc: 96.25%
Epoch 009 | Loss: 0.0062 | Acc: 97.50%
Epoch 010 | Loss: 0.0057 | Acc: 96.25%
Epoch 011 | Loss: 0.0053 | Acc: 96.25%
Epoch 012 | Loss: 0.0047 | Acc: 97.50%
Epoch 013 | Loss: 0.0043 | Acc: 97.50%
Epoch 014 | Loss: 0.0042 | Acc: 96.25%
Epoch 015 | Loss: 0.0046 | Acc: 97.50%
Epoch 016 | Loss: 0.0040 | Acc: 97.50%
Epoch 017 | Loss: 0.0045 | Acc: 97.50%
Epoch 018 | Loss: 0.0046 | Acc: 97.50%
Epoch 019 | Loss: 0.0041 | Acc: 97.50%
Epoch 020 | Loss: 0.0040 | Acc: 97.50%
Epoch 021 | Loss: 0.0040 | Acc: 97.50%
Epoch 022 | Loss: 0.0048 | Acc: 97.50%
Epoch 023 | Loss: 0.0046 | Acc: 97.50%
Epoch 024 | Loss: 0.0044 | Acc: 97.50%
Epoch 025 | Loss: 0.0042 | Acc: 97.50%
Epoch 026 | Loss: 0.0042 